# Ising model 

When dealing with binary events, the Gaussian distribution is (at least) an unreasonable choice. Instead, a multivariate Bernoulli distribution with a properly accounted for graphical structure works best. 

This notebook considers the model 

$$
f(p_{1}, \dots, p_{N} | x_{1}, \dots, x_{N}) \propto \prod_{1 \le n, m \le N} (p_{n}p_{m})^{\frac{x_{n}x_{m}}{2} \cdot \sigma_{nm}}, 
$$

in which $x_{n}, x_{m} \in \{-1, 1\}$. Alternatively, this may be written as 

$$
f(p_{1}, \dots, p_{N} | x_{1}, \dots, x_{N}) \propto \prod_{1 \le n \le N} p_{n}^{\sum_{1 \le m \le N} \frac{x_{n}x_{m}}{2} \sigma_{n, m}}, 
$$

which (in contrast to its Gaussian counterpart) can be easily interpreted. The normalizing factor is 

$$
\int_{p_{1}, \dots, p_{N}} f(p_{1}, \dots, p_{N} | x_{1}, \dots, x_{N}) \mathrm{d}p_{1} \dots \mathrm{d}p_{N} = \prod_{n} \int_{[0, 1]} p_{n}^{\sum_{m} \frac{x_{n}x_{m}}{2}\sigma_{nm}} \mathrm{d} p_{n} = \prod_{n} \frac{1}{1 + \sum_{m} \frac{x_{n} x_{m}}{2} \sigma_{nm}}. 
$$

This assumes that the function is integrable, i.e., that the above summation is larger than -1. This might be achived by truncation. From a probabilistic perspective, this may be perceived as "the minimum correlation" between a collection of elements. 

In [8]:
import torch
import sys
import scipy.stats

sys.path.append("..")

from mvn_torch import mvn_cdf_torch

In [ ]:
def bernoulli_cdf(p: torch.Tensor, cov: torch.Tensor, x: torch.Tensor):
    x = x.reshape(-1, 1) @ x.reshape(1, -1) # Outer product
    sigma = cov * x / 2
    sigma_sum = 1 + sigma.sum(dim=1)
    sigma_sum_log = torch.log(sigma_sum)
    
    log_z = torch.logsumexp(sigma_sum_log, dim=0)

    log_p = torch.logsumexp(
        torch.log(p) * sigma_sum, dim=0
     ) - log_z

    return torch.exp(log_p)

In [ ]:
p = torch.tensor([0.5, 0.5])
pho = 0.95
cov = torch.tensor([[1, pho], [pho, 1]])
x = torch.tensor([1, -1])

bernoulli_cdf(p, cov, x)

tensor(0.4794)

In [ ]:
a = scipy.stats.norm.ppf(p.numpy())
a = torch.tensor(a)
mvn_cdf_torch(a, torch.zeros_like(p), torch.cholesky(cov))

tensor(0.1665)